# Time series forecasting

In this notebook, we will use LSTMs for weather forecasting.

The notebook is based on this TensorFlow tutorial: https://www.tensorflow.org/tutorials/structured_data/time_series. However, here we will use [PyTorch](https://pytorch.org/) to implement our model.

The dataset we use here was recorded by the [Max Planck Institute for Biogeochemistry](https://www.bgc-jena.mpg.de/). This dataset contains 14 different features such as air temperature, atmospheric pressure, and humidity. These were collected every 10 minutes, beginning in 2003. For efficiency, you will use only the data collected between 2009 and 2016. This section of the dataset was prepared by François Chollet for his book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python).

In [ ]:
import os
import random

import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
from tqdm import tqdm

In [ ]:
# Download the data
import requests
import zipfile
response = requests.get('https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip')
open('weather_dataset.zip', 'wb').write(response.content)
with zipfile.ZipFile('weather_dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('./')
df = pd.read_csv('jena_climate_2009_2016.csv')

In [ ]:
# Let's take a look at the data frame
df

The dataset contains measurements taken each 10 minutes. We will subsample it to have hourly measurements.

In [ ]:
# Slice [start:stop:step], starting from index 5 take every 6th record.
df = df[5::6]

date_time = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')

In [ ]:
plot_cols = ['T (degC)', 'p (mbar)', 'rho (g/m**3)']
plot_features = df[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

plot_features = df[plot_cols][:480]
plot_features.index = date_time[:480]
_ = plot_features.plot(subplots=True)

Let's look at some statistics of our data...

In [ ]:
df.describe().transpose()

In [ ]:
# Remove invalid values
wv = df['wv (m/s)']
bad_wv = (wv == -9999.0)
wv[bad_wv] = 0.0

max_wv = df['max. wv (m/s)']
bad_max_wv = (max_wv == -9999.0)
max_wv[bad_max_wv] = 0.0

# The above inplace edits are reflected in the DataFrame.
df['wv (m/s)'].min()

Our dataset contains angular data for the wind direction. However, angles are not suitable representations as model inputs (not that 0 and 360 degrees are the same). Moreover, whenever the wind speed is zero, the wind direction is meaningless.

Therefore, we will use the wind speed and direction to build the corresponding **wind vector**, which will be one of the inputs of our model.

In [ ]:
# Build wind vector
wv = df.pop('wv (m/s)')
max_wv = df.pop('max. wv (m/s)')

# Convert to radians.
wd_rad = df.pop('wd (deg)')*np.pi / 180

# Calculate the wind x and y components.
df['Wx'] = wv*np.cos(wd_rad)
df['Wy'] = wv*np.sin(wd_rad)

# Calculate the max wind x and y components.
df['max Wx'] = max_wv*np.cos(wd_rad)
df['max Wy'] = max_wv*np.sin(wd_rad)

Date strings are also innapropriate as model inputs. Can you understand the preprocessing we are doing below?

In [ ]:
timestamp_s = date_time.map(pd.Timestamp.timestamp)
day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

plt.plot(np.array(df['Day sin'])[:25])
plt.plot(np.array(df['Day cos'])[:25])
plt.xlabel('Time [h]')
plt.title('Time of day signal')
plt.show()

**Explain** the preprocessing above in your own words:

**YOUR ANSWER HERE**

Now, we are ready to split our data into train, validation and test splits. We will use the first 70% samples for training, the following 20% for validation and the last 10% for testing.

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

Note that we are **not** splitting our data randomly. Can you explain why a random split is not a good idea in this case?

**YOUR ANSWER HERE**

It is crucial to scale features before feeding them through a neural network. Normalize the data in the cell below by subtracting the mean and dividing by the standard deviation.

You should create the variables `train_mean` and `train_std` and use them to normalize the three data splits.

In [ ]:
## YOUR CODE HERE ##

## *** ##

In [ ]:
# Look at normalized data
df_std = (df - train_mean) / train_std
df_std = df_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(df.keys(), rotation=90)

The `torch.utils.data.Dataset` class provides a very convenient way of storing and loading data. In the cell below, we define the class `WeatherData`, which inherits from this `Dataset` class. The following functions must be defined within the class:
- `__init__`, where the attributes of the class are defined;
- `__getitem__`, which gets an index (`int`) and returns the data example corresponding to the provided index (can be a `tuple` or a `dict`);
- `__len__`, which returns the length of the dataset.

We have already written the functions `__getitem__` and `__len__` for you. All you need to do is write the code for the `__init__` function. There, you should create two lists as attributes of the class: `inputs` and `targets`. The `i`-th element of the list `inputs` should be a `torch.FloatTensor` containing a slice of the data with `input_width` consecutive samples. The `i`-th element of the list `targets` should be a `torch.FloatTensor` containing a slice of the data with the `target_width` consecutive samples that follow after `inputs[i]`.

In [ ]:
class WeatherData(torch.utils.data.Dataset):
    def __init__(self, df, input_width, target_width):
        ## YOUR CODE HERE ##
        
        ## *** ##               
    def __getitem__(self, idx):
        return {'inputs': self.inputs[idx],
                'targets': self.targets[idx]}
    
    def __len__(self):
        return min(len(self.inputs), len(self.targets))

We will use the observations of the previous 24 hours to predict the temperature for the next 24 hours. Thus, we will set both `input_width` and `target_width` to 24 in the instances of the dataset. After you complete this exercise, you can try with different values.

In [ ]:
INPUT_WIDTH = 24
TARGET_WIDTH = 24

train_data = WeatherData(train_df, INPUT_WIDTH, TARGET_WIDTH)
val_data = WeatherData(val_df, INPUT_WIDTH, TARGET_WIDTH)
test_data = WeatherData(test_df, INPUT_WIDTH, TARGET_WIDTH)

In [ ]:
# Let's visualize some examples
idx = random.randint(0, len(train_data))
inputs = train_data[idx]['inputs'].numpy()
targets = train_data[idx]['targets'].numpy()

for i in range(len(train_df.columns)):
    plt.plot(range(INPUT_WIDTH), inputs[:,i], label='input')
    plt.plot(range(INPUT_WIDTH, INPUT_WIDTH + TARGET_WIDTH), targets[:,i], label='target')
    plt.ylabel(train_df.columns[i])
    plt.xlabel('Time')
    plt.legend()
    plt.show()

Now, it's time to define our neural network! It will consist of a `torch.nn.LSTMCell` with a state dimension defined by `hidden_size`, followed by a `torch.nn.Linear` layer that projects from the hidden space to the output.

The best way to define a neural network in PyTorch is by extending the class `torch.nn.Module`. For this purpose, you need to define two functions:
- `__init__`, where all the layers of the network are instantiated;
- `forward`, where the forward pass is defined.

Complete the two methods in the cell below. The comments throughout the code will provide you some hints.

In [ ]:
class WeatherPredictor(torch.nn.Module):
    def __init__(self, num_features, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        # instantiate a torch.nn.LSTMCell and a torch.nn.Linear layer
        ## YOUR CODE HERE ##
        
        ## *** ##
        
    def forward(self, inputs, num_steps):
        inputs = inputs.transpose(0, 1)  # batch, time, dim -> time, batch, dim
        
        # we initialize the internal states of the LSTM with zeros
        hx = torch.zeros((inputs.shape[1], self.hidden_size), device=inputs.device)
        cx = torch.zeros((inputs.shape[1], self.hidden_size), device=inputs.device)
        
        # warmup: feed the inputs through the LSTM one by one from t = 0 up to t = T-1
        ## YOUR CODE HERE ##
        
        ## *** ##
        
        # predict num_steps into the future autoregressively
        outputs = []
        input_t = inputs[-1]
        ## YOUR CODE HERE ##
        
        ## *** ##
        
        # stack the list of outputs into a single tensor
        outputs = torch.stack(outputs)
        
        outputs = outputs.transpose(0, 1)  # time, batch, dim -> batch, time, dim
        return outputs
        

Now we are going to define the evaluation and training loops of our model in the functions `evaluate` and `fit`, respectively. This should be identical to what you may have done before for non-recurrent models.

In [ ]:
def evaluate(model, data_loader, **kwargs):
    loss_fn = kwargs.get('loss_fn', torch.nn.functional.mse_loss)
    device = kwargs.get('device', torch.device('cpu'))
    
    model.eval()
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    avg_loss = 0.
    for i, batch in pbar:
        ## YOUR CODE HERE ##
        
        ## *** ##
        pbar.set_description(f'loss = {loss:.3f}')
    avg_loss /= len(val_loader)
    return avg_loss

def fit(model, train_loader, val_loader, optimizer, **kwargs):
    num_epochs = kwargs.get('num_epochs', 100)
    loss_fn = kwargs.get('loss_fn', torch.nn.functional.mse_loss)
    device = kwargs.get('device', torch.device('cpu'))
    
    train_loss_hist, val_loss_hist = [], []
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        
        print('Training phase...')
        model.train()
        train_loss = 0.
        pbar = tqdm(enumerate(train_loader), total=len(train_loader))
        for i, batch in pbar:
            ## YOUR CODE HERE ##
            
            ## *** ##
            train_loss += loss.item()
            pbar.set_description(f'loss = {loss:.3f}')
        train_loss /= len(train_loader)
        print(f'train loss = {train_loss:.3f}')
        train_loss_hist.append(train_loss)
        
        print('Validation phase...')
        val_loss = evaluate(model, val_loader, loss_fn=loss_fn, device=device)
        print(f'validation loss = {val_loss:.3f}')
        val_loss_hist.append(val_loss)
        
    return train_loss_hist, val_loss_hist
        

The only thing that is still missing are the `torch.utils.data.DataLoader`s. These provide a very convenient way of aggregating the data into mini-batches. We have already defined them for you in the cell below.

We have also defined a set of hyperparameters (`HIDDEN_SIZE`, `LEARNING_RATE`, `NUM_EPOCHS`, and `BATCH_SIZE`). You should understand the role of each of these and you can experiment with different values later on.

The code is also prepared to run using CUDA if you have a CUDA-capable GPU in your computer. The commented code is for running on Mac M1 GPUs. However, as of today, the training will be faster on Mac M1 CPU than on the GPU.

In [ ]:
HIDDEN_SIZE = 32
LEARNING_RATE = 1e-3
NUM_EPOCHS = 100
BATCH_SIZE = 16

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
# elif torch.backends.mps.is_available():
#     DEVICE = torch.device('mps')
else:
    DEVICE = torch.device('cpu')
print('DEVICE:', DEVICE)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

model = WeatherPredictor(train_data[0]['inputs'].shape[1], HIDDEN_SIZE).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

Train the model by running the cell below. It may take a while depending on your hardware and on the `NUM_EPOCHS` that you have chosen.

In [ ]:
train_loss, val_loss = fit(model, train_loader, val_loader, optimizer, num_epochs=NUM_EPOCHS, device=DEVICE)

In [ ]:
plt.title('Train and validation losses')
plt.plot(range(len(train_loss)), train_loss, label='train loss')
plt.plot(range(len(val_loss)), val_loss, label='validation loss')
plt.legend()
plt.show()

Save the trained model to a file.

In [ ]:
torch.save(model.state_dict(), 'model.pt')

Compute the loss on the test set.

In [ ]:
test_loss = evaluate(model, val_loader, device=DEVICE)
print(f'Test loss = {test_loss:.3f}')

Now you will implement a `predict` function, which returns a `list` with the predictions of your model for each of the examples in the `data_loader`. This should be very similar to the `evaluate` function that you've written before, except that now there is no loss to be computed.

In [ ]:
def predict(model, data_loader, **kwargs):
    num_steps = kwargs.get('num_steps', 24)
    device = kwargs.get('device', torch.device('cpu'))
    
    model.eval()
    preds = []
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for i, batch in pbar:
        ## YOUR CODE HERE ##
        
        ## *** ##
    return preds

In [ ]:
test_preds = predict(model, test_loader, num_steps=TARGET_WIDTH, device=DEVICE)

Finally, let's visualize a few predictions of our model. You can run the cell below multiple times to visualize the temperature predictions for different examples.

In [ ]:
idx = random.randint(0, len(test_data))
inputs = test_data[idx]['inputs'].numpy()
targets = test_data[idx]['targets'].numpy()
preds = test_preds[idx].numpy()

for i in [1, 2, 3]:
    plt.plot(range(INPUT_WIDTH), inputs[:,i], label='input')
    plt.plot(range(INPUT_WIDTH, INPUT_WIDTH + TARGET_WIDTH), targets[:,i], label='target')
    plt.plot(range(INPUT_WIDTH, INPUT_WIDTH + TARGET_WIDTH), preds[:,i], label='prediction')
    plt.ylabel(test_df.columns[i])
    plt.xlabel('Time')
    plt.legend()
    plt.show()

## Playground

Now, it's time for you to be creative! Try to find ways of improving the model. You can start by playing around with hyperparameters and then going for more complex stuff, like adding more layers or even trying with a different architecture. Perhaps, on this problem, a convolutional network works better than a recurrent one, who knows?

**Don't be afraid to try! Good luck! :)**